In [3]:
import pandas as pd
df=pd.read_csv("hyderabad_real_estate.csv")

In [4]:
df

,Name,Location,BHK,Type,Price,Size,Status,Delivery Date,Listed By
0,Fortune Butterfly City,Kadthal,NaN,NaN,₹23.99 L,NaN,NaN,"May, 2015",FORTUNE BUTTERFLY CITY
1,The Cascades Neopolis,Kokapet,5 BHK,Flat,₹32.03 L,4825 sq.ft,Under Construction,"Mar, 2030",Seller + 1 more
2,Sreenidhi Neocity,Sadasivpet,NaN,NaN,₹35.99 L,NaN,NaN,"Jun, 2026",GHR Lakshmi Infra and Urban Blocks Realty
3,Sri Tirumala Millennium Phase III,Mallapur,3 BHK,Flat,₹41.99 L,1521 sq.ft,Under Construction,"Oct, 2027",Developer + 1 more
4,Sri Jagathswapna Spanesta Villa,Bacharam,5 BHK,Villa,₹47.99 L,5500 sq.ft,Under Construction,"Apr, 2026",Ananthaa Spaces
...,...,...,...,...,...,...,...,...,...
527,Primark Eco Nest,Kompally,3 BHK,Flat,₹34.1 L,2165 sq.ft,Under Construction,"Oct, 2027",Seller + 1 more
528,Poulomi Palazzo,Kokapet,4 BHK,Flat,₹36.11 L,10160 sq.ft,Under Construction,"Mar, 2028",Lakshinfraprojects
529,Sage By Raghava,Kollur,4 BHK,Villa,₹36.4 L,6370 sq.ft,Under Construction,"Aug, 2029",Seller
530,Sew Estella,Kondapur,4 BHK,Flat,₹36.42 L,3160 sq.ft,Ready to Move,"Jul, 2015",MAGNA SOLITAIRE


In [5]:
df['Listed By'] = df['Listed By'].str.replace(r"\+\s*\d+\s*more", "", regex=True).str.strip()

In [6]:
df.isnull().sum()

Name               0
Location           0
BHK              103
Type             102
Price              0
Size             104
Status           104
Delivery Date      0
Listed By          0
dtype: int64

In [7]:
df['BHK']=df['BHK'].fillna(df['BHK'].mode()[0])

In [8]:
df['BHK'] = df['BHK'].str.extract(r'(\d+)').astype('Int64')

In [9]:
df["Type"]=df["Type"].fillna(df["Type"].mode()[0])

In [10]:
df["Status"]=df["Status"].fillna(df["Status"].mode()[0])

In [11]:
df['Size'] = df['Size'].str.replace("sq.ft","", regex=False).str.strip()

In [12]:
df['Delivery_Month'] = df['Delivery Date'].str.extract(r'([A-Za-z]+)')
df['Delivery_Year'] = df['Delivery Date'].str.extract(r'(\d{4})').astype('Int64')

In [13]:
df['Size'] = pd.to_numeric(df['Size'], errors='coerce')
# Group by BHK and get average size
avg_size_per_bhk = df.groupby('BHK')['Size'].mean().round()
# Fill missing values using BHK-wise average
df['Size'] = df.apply(
    lambda row: avg_size_per_bhk[row['BHK']] if pd.isna(row['Size']) and row['BHK'] in avg_size_per_bhk else row['Size'],
    axis=1)

In [14]:
df.isnull().sum()

Name              0
Location          0
BHK               0
Type              0
Price             0
Size              0
Status            0
Delivery Date     0
Listed By         0
Delivery_Month    0
Delivery_Year     0
dtype: int64

In [15]:
def convert_price_to_lakhs(price):
    price = price.replace("₹", "").replace(",", "").strip() 
    # Convert crores to lakhs
    if 'Cr' in price:
        price = price.replace('Cr', '')
        return float(price) * 100
    elif 'L' in price:
        price = price.replace('L', '')
        return float(price)
df['Price'] = df['Price'].apply(convert_price_to_lakhs)

In [16]:
df.rename(columns={
    'Size': 'Size (in sqft)',
    'Price': 'Price (in lakhs)'
}, inplace=True)

In [17]:
df

,Name,Location,BHK,Type,Price (in lakhs),Size (in sqft),Status,Delivery Date,Listed By,Delivery_Month,Delivery_Year
0,Fortune Butterfly City,Kadthal,3,Flat,23.99,2093.0,Under Construction,"May, 2015",FORTUNE BUTTERFLY CITY,May,2015
1,The Cascades Neopolis,Kokapet,5,Flat,32.03,4825.0,Under Construction,"Mar, 2030",Seller,Mar,2030
2,Sreenidhi Neocity,Sadasivpet,3,Flat,35.99,2093.0,Under Construction,"Jun, 2026",GHR Lakshmi Infra and Urban Blocks Realty,Jun,2026
3,Sri Tirumala Millennium Phase III,Mallapur,3,Flat,41.99,1521.0,Under Construction,"Oct, 2027",Developer,Oct,2027
4,Sri Jagathswapna Spanesta Villa,Bacharam,5,Villa,47.99,5500.0,Under Construction,"Apr, 2026",Ananthaa Spaces,Apr,2026
...,...,...,...,...,...,...,...,...,...,...,...
527,Primark Eco Nest,Kompally,3,Flat,34.10,2165.0,Under Construction,"Oct, 2027",Seller,Oct,2027
528,Poulomi Palazzo,Kokapet,4,Flat,36.11,10160.0,Under Construction,"Mar, 2028",Lakshinfraprojects,Mar,2028
529,Sage By Raghava,Kollur,4,Villa,36.40,6370.0,Under Construction,"Aug, 2029",Seller,Aug,2029
530,Sew Estella,Kondapur,4,Flat,36.42,3160.0,Ready to Move,"Jul, 2015",MAGNA SOLITAIRE,Jul,2015


In [20]:
df.to_csv('cleaned_Hyderabad_real_estate.csv', index=False)

In [23]:
%pwd

'C:\\Users\\Dell\\Eda'